In [1]:
# pandas读写csv文件
import pandas as pd
import numpy as np

from datetime import datetime


def log_print(*arg, log=True):
    if log:
        print(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),
              *arg,
              flush=True)


path = '/Users/mingjue/downloads/'

df_train_1 = pd.read_csv(path + 'data_format1/train_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_test_1 = pd.read_csv(path + 'data_format1/test_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_user_info = pd.read_csv(path + 'data_format1/user_info_format1.csv', dtype='str')
df_user_log = pd.read_csv(path + 'data_format1/user_log_format1.csv', dtype='str')
df_train_2 = pd.read_csv(path + 'data_format2/train_format2.csv', dtype='str')
df_test_2 = pd.read_csv(path + 'data_format2/test_format2.csv', dtype='str')
df_infer = pd.read_csv(path + 'sample_submission.csv', dtype='str')

# df_user_info
# age_range 和 gender 有缺失值
# age_range 的取值为0-8，0表示缺失值（age_range一般对应年龄大小顺序，根据分布判断0表示未知）
# gender 的取值为0-2，2表示缺失值(根据官网描述，2和NULL表示未知)
# 将df_user_info的 age_range 列  字段转换为float， 取值0都替换为空值,
df_user_info['age_range'] = df_user_info['age_range'].astype(float)
df_user_info['age_range'] = df_user_info['age_range'].replace(0, np.nan)
# 将df_user_info的 gender 列 中 取值为2替换为空值
df_user_info['gender'] = df_user_info['gender'].replace('2', np.nan)

# df_user_log
df_user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)
# time_stamp 的取值为0511到1112共186天，表示mmdd，没有年份，请转换为标准日期格式
df_user_log['time_stamp'] = pd.to_datetime('2025' + df_user_log['time_stamp'], format='%Y%m%d')
df_user_log['recency'] = (pd.to_datetime('2025-11-12') - df_user_log['time_stamp']).dt.days + 1

# 计算用户rfm, 数据没有monetary，所以不能计算rfm，只能计算rf
# recency：距离最后一次消费的天数
# frequency：消费的次数



In [ ]:
df_user_rf = df_user_log.groupby('user_id').agg(
    r=('recency', 'min'),
    f=('user_id', 'count'), 
    f_item=('item_id', 'nunique'),
    f_cat=('cat_id', 'nunique'), 
    f_merchant=('merchant_id', 'nunique'),
    f_brand=('brand_id', 'nunique'),
).reset_index()
log_print('df_user_rf shape:', df_user_rf.shape)
df_user_rf_0 = df_user_log.query('"0"==action_type').groupby('user_id').agg(
    r_0=('recency', 'min'),
    f_0=('user_id', 'count'), 
    f_item_0=('item_id', 'nunique'),
    f_cat_0=('cat_id', 'nunique'), 
    f_merchant_0=('merchant_id', 'nunique'),
    f_brand_0=('brand_id', 'nunique'),
).reset_index() 
log_print('df_user_rf_0 shape:', df_user_rf_0.shape)
df_user_rf_1 = df_user_log.query('"1"==action_type').groupby('user_id').agg(
    r_1=('recency', 'min'),
    f_1=('user_id', 'count'), 
    f_item_1=('item_id', 'nunique'),
    f_cat_1=('cat_id', 'nunique'), 
    f_merchant_1=('merchant_id', 'nunique'),
    f_brand_1=('brand_id', 'nunique'),
).reset_index()
log_print('df_user_rf_1 shape:', df_user_rf_1.shape)
df_user_rf_2 = df_user_log.query('"2"==action_type').groupby('user_id').agg(
    r_2=('recency', 'min'),
    f_2=('user_id', 'count'), 
    f_item_2=('item_id', 'nunique'),
    f_cat_2=('cat_id', 'nunique'), 
    f_merchant_2=('merchant_id', 'nunique'),
    f_brand_2=('brand_id', 'nunique'),
).reset_index() 
log_print('df_user_rf_2 shape:', df_user_rf_2.shape)
df_user_rf_3 = df_user_log.query('"3"==action_type').groupby('user_id').agg(
    r_3=('recency', 'min'),
    f_3=('user_id', 'count'), 
    f_item_3=('item_id', 'nunique'),
    f_cat_3=('cat_id', 'nunique'), 
    f_merchant_3=('merchant_id', 'nunique'),
    f_brand_3=('brand_id', 'nunique'),
).reset_index()
log_print('df_user_rf_3 shape:', df_user_rf_3.shape)
df_user_rf = df_user_rf.merge(
    df_user_rf_0, how='left', on='user_id',
).merge(
    df_user_rf_1, how='left', on='user_id',
).merge(
    df_user_rf_2, how='left', on='user_id',
).merge(
    df_user_rf_3, how='left', on='user_id',
)
log_print('df_user_rf shape:', df_user_rf.shape)
del df_user_rf_0, df_user_rf_1, df_user_rf_2, df_user_rf_3,
# 计算 user_id, merchant_id 交互特征
df_user_merchant_rf = df_user_log.groupby(['user_id', 'merchant_id']).agg(
    r_um=('recency', 'min'),
    f_um=('user_id', 'count'),
    f_item_um=('item_id', 'nunique'),
    f_cat_um=('cat_id', 'nunique'),
    f_brand_um=('brand_id', 'nunique'),
).reset_index()
log_print('df_user_merchant_rf shape:', df_user_merchant_rf.shape)
df_user_merchant_rf_0 = df_user_log.query('"0"==action_type').groupby(['user_id', 'merchant_id']).agg(
    r_um_0=('recency', 'min'),
    f_um_0=('user_id', 'count'),        
    f_item_um_0=('item_id', 'nunique'), 
    f_cat_um_0=('cat_id', 'nunique'),   
    f_brand_um_0=('brand_id', 'nunique'),       
).reset_index()
log_print('df_user_merchant_rf_0 shape:', df_user_merchant_rf_0.shape)
df_user_merchant_rf_1 = df_user_log.query('"1"==action_type').groupby(['user_id', 'merchant_id']).agg(
    r_um_1=('recency', 'min'),
    f_um_1=('user_id', 'count'),        
    f_item_um_1=('item_id', 'nunique'), 
    f_cat_um_1=('cat_id', 'nunique'),   
    f_brand_um_1=('brand_id', 'nunique'),           
).reset_index()
log_print('df_user_merchant_rf_1 shape:', df_user_merchant_rf_1.shape)
df_user_merchant_rf_2 = df_user_log.query('"2"==action_type').groupby(['user_id', 'merchant_id']).agg(
    r_um_2=('recency', 'min'),
    f_um_2=('user_id', 'count'),        
    f_item_um_2=('item_id', 'nunique'), 
    f_cat_um_2=('cat_id', 'nunique'),   
    f_brand_um_2=('brand_id', 'nunique'),           
).reset_index()
log_print('df_user_merchant_rf_2 shape:', df_user_merchant_rf_2.shape)
df_user_merchant_rf_3 = df_user_log.query('"3"==action_type').groupby(['user_id', 'merchant_id']).agg(
    r_um_3=('recency', 'min'),
    f_um_3=('user_id', 'count'),        
    f_item_um_3=('item_id', 'nunique'), 
    f_cat_um_3=('cat_id', 'nunique'),   
    f_brand_um_3=('brand_id', 'nunique'),               
).reset_index()
log_print('df_user_merchant_rf_3 shape:', df_user_merchant_rf_3.shape)
df_user_merchant_rf = df_user_merchant_rf.merge(
    df_user_merchant_rf_0, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_1, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_2, how='left', on=['user_id', 'merchant_id'],
).merge(
    df_user_merchant_rf_3, how='left', on=['user_id', 'merchant_id'],
)
log_print('df_user_merchant_rf shape:', df_user_merchant_rf.shape)
del df_user_merchant_rf_0, df_user_merchant_rf_1, df_user_merchant_rf_2, df_user_merchant_rf_3,
# 计算 merchant_id 特征
df_merchant = df_user_log.groupby('merchant_id').agg(
    r_m=('recency', 'min'),
    f_m=('user_id', 'count'),
    f_user_m=('user_id', 'nunique'),
    f_item_m=('item_id', 'nunique'),
    f_cat_m=('cat_id', 'nunique'),
    f_brand_m=('brand_id', 'nunique'),
).reset_index()
log_print('df_merchant shape:', df_merchant.shape)
df_merchant_0 = df_user_log.query('"0"==action_type').groupby('merchant_id').agg(
    r_m_0=('recency', 'min'),
    f_m_0=('user_id', 'count'),
    f_user_m_0=('user_id', 'nunique'),
    f_item_m_0=('item_id', 'nunique'),
    f_cat_m_0=('cat_id', 'nunique'),
    f_brand_m_0=('brand_id', 'nunique'),
).reset_index()
log_print('df_merchant_0 shape:', df_merchant_0.shape)
df_merchant_1 = df_user_log.query('"1"==action_type').groupby('merchant_id').agg(
    r_m_1=('recency', 'min'),
    f_m_1=('user_id', 'count'),
    f_user_m_1=('user_id', 'nunique'),
    f_item_m_1=('item_id', 'nunique'),
    f_cat_m_1=('cat_id', 'nunique'),
    f_brand_m_1=('brand_id', 'nunique'),    
).reset_index()
log_print('df_merchant_1 shape:', df_merchant_1.shape)
df_merchant_2 = df_user_log.query('"2"==action_type').groupby('merchant_id').agg(
    r_m_2=('recency', 'min'),
    f_m_2=('user_id', 'count'),
    f_user_m_2=('user_id', 'nunique'),
    f_item_m_2=('item_id', 'nunique'),
    f_cat_m_2=('cat_id', 'nunique'),
    f_brand_m_2=('brand_id', 'nunique'),    
).reset_index()
log_print('df_merchant_2 shape:', df_merchant_2.shape)
df_merchant_3 = df_user_log.query('"3"==action_type').groupby('merchant_id').agg(
    r_m_3=('recency', 'min'),
    f_m_3=('user_id', 'count'),
    f_user_m_3=('user_id', 'nunique'),
    f_item_m_3=('item_id', 'nunique'),
    f_cat_m_3=('cat_id', 'nunique'),
    f_brand_m_3=('brand_id', 'nunique'),
).reset_index()
log_print('df_merchant_3 shape:', df_merchant_3.shape)
df_merchant = pd.merge(df_merchant, df_merchant_0, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_1, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_2, on='merchant_id', how='left')
df_merchant = pd.merge(df_merchant, df_merchant_3, on='merchant_id', how='left')
log_print('df_merchant shape:', df_merchant.shape)
del df_merchant_0, df_merchant_1, df_merchant_2, df_merchant_3,
df_user_rf.to_parquet(path +'user_rf.parquet')
df_user_merchant_rf.to_parquet(path +'user_merchant_rf.parquet')
df_merchant.to_parquet(path +'merchant.parquet')

2025-10-13 16:58:43.543946 df_user_rf shape: (424170, 7)
2025-10-13 17:01:12.638821 df_user_rf_0 shape: (423862, 7)
2025-10-13 17:01:14.177730 df_user_rf_1 shape: (31044, 7)
2025-10-13 17:01:22.688616 df_user_rf_2 shape: (424170, 7)
2025-10-13 17:01:29.348064 df_user_rf_3 shape: (233428, 7)
2025-10-13 17:01:29.885844 df_user_rf shape: (424170, 31)
2025-10-13 17:03:49.311226 df_user_merchant_rf shape: (14058666, 7)
2025-10-13 17:05:58.476718 df_user_merchant_rf_0 shape: (13263204, 7)
2025-10-13 17:06:00.088527 df_user_merchant_rf_1 shape: (62236, 7)
2025-10-13 17:06:07.756090 df_user_merchant_rf_2 shape: (2209433, 7)
2025-10-13 17:06:13.484044 df_user_merchant_rf_3 shape: (1972052, 7)
2025-10-13 17:06:33.234049 df_user_merchant_rf shape: (14058666, 27)
2025-10-13 17:09:37.837622 df_merchant shape: (4995, 7)
2025-10-13 17:12:30.219396 df_merchant_0 shape: (4995, 7)
2025-10-13 17:12:31.650152 df_merchant_1 shape: (4729, 7)
2025-10-13 17:12:39.664247 df_merchant_2 shape: (4995, 7)


In [3]:
# df_infer = pd.read_csv(path + 'sample_submission.csv')
# # 简单的预测，随机大于0小于1的小数，比如0.5、0.8，均值0.2
# df_infer['prob'] = np.random.rand(len(df_infer))
# # 保存结果到prediction.csv文件中
# df_infer.to_csv(path + 'prediction.csv', index=False)

In [ ]:
df_user_rf = pd.read_parquet(path +'user_rf.parquet')
df_user_merchant_rf = pd.read_parquet(path +'user_merchant_rf.parquet')
df_merchant = pd.read_parquet(path +'merchant.parquet')

df_train = df_train_1.merge(
    df_user_info, on='user_id', how='left',
).merge(
    df_user_merchant_rf, on=['user_id', 'merchant_id'], how='left',
).merge(
    df_user_rf, on='user_id', how='left',
).merge(
    df_merchant, on='merchant_id', how='left',
)
df_train['gender'] = df_train['gender'].astype(str)

df_test = df_test_1.merge(
    df_user_info, on='user_id', how='left',
).merge(
    df_user_merchant_rf, on=['user_id', 'merchant_id'], how='left',
).merge(
    df_user_rf, on='user_id', how='left',
).merge(
    df_merchant, on='merchant_id', how='left',
)
df_test['gender'] = df_test['gender'].astype(str)
# 将所有的f开头字段缺失补零 
cols_f = [
    'f', 'f_item', 'f_cat', 'f_merchant', 'f_brand',
    'f_0', 'f_item_0', 'f_cat_0', 'f_merchant_0', 'f_brand_0',
    'f_1', 'f_item_1', 'f_cat_1', 'f_merchant_1', 'f_brand_1',
    'f_2', 'f_item_2', 'f_cat_2', 'f_merchant_2', 'f_brand_2',
    'f_3', 'f_item_3', 'f_cat_3', 'f_merchant_3', 'f_brand_3',
    'f_um', 'f_item_um', 'f_cat_um', 'f_brand_um',
    'f_um_0', 'f_item_um_0', 'f_cat_um_0', 'f_brand_um_0',
    'f_um_1', 'f_item_um_1', 'f_cat_um_1', 'f_brand_um_1',
    'f_um_2', 'f_item_um_2', 'f_cat_um_2', 'f_brand_um_2',
    'f_um_3', 'f_item_um_3', 'f_cat_um_3', 'f_brand_um_3',  
    'f_m', 'f_user_m', 'f_item_m', 'f_cat_m', 'f_brand_m',
    'f_m_0', 'f_user_m_0', 'f_item_m_0', 'f_cat_m_0', 'f_brand_m_0',
    'f_m_1', 'f_user_m_1', 'f_item_m_1', 'f_cat_m_1', 'f_brand_m_1',
    'f_m_2', 'f_user_m_2', 'f_item_m_2', 'f_cat_m_2', 'f_brand_m_2',
    'f_m_3', 'f_user_m_3', 'f_item_m_3', 'f_cat_m_3', 'f_brand_m_3',   
]
df_train[cols_f].fillna(0, inplace=True)
df_test[cols_f].fillna(0, inplace=True)
for i in range(4):
    for col in ['', '_item', '_cat', '_merchant', '_brand']:
        df_train[f'fp{col}_{i}'] = df_train[f'f{col}_{i}'] / (df_train[f'f{col}'] + 1e-5)
        df_test[f'fp{col}_{i}'] = df_test[f'f{col}_{i}'] / (df_test[f'f{col}'] + 1e-5)
    for col in ['', '_item', '_cat', '_brand']:
        df_train[f'fp{col}_um_{i}'] = df_train[f'f{col}_um_{i}'] / (df_train[f'f{col}_um'] + 1e-5)
        df_test[f'fp{col}_um_{i}'] = df_test[f'f{col}_um_{i}'] / (df_test[f'f{col}_um'] + 1e-5)
    for col in ['', '_user', '_item', '_cat', '_brand']:
        df_train[f'fp{col}_m_{i}'] = df_train[f'f{col}_m_{i}'] / (df_train[f'f{col}_m'] + 1e-5)
        df_test[f'fp{col}_m_{i}'] = df_test[f'f{col}_m_{i}'] / (df_test[f'f{col}_m'] + 1e-5)

/var/folders/xk/njww8w8d7_56yzx17rd8c3rr0000gn/T/ipykernel_69107/2037182088.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[cols_f].fillna(0, inplace=True)
/var/folders/xk/njww8w8d7_56yzx17rd8c3rr0000gn/T/ipykernel_69107/2037182088.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[cols_f].fillna(0, inplace=True)


In [22]:
# df_train catboost 二分类 训练推理
# 固定随机种子，保证结果可复现
np.random.seed(42)
# 训练代码 
from catboost import CatBoostClassifier, Pool  
from sklearn.model_selection import train_test_split
features = [
    'age_range', 'gender',
    'r', 'f', 'f_item', 'f_cat', 'f_merchant', 'f_brand',
    'r_0', 'f_0', 'f_item_0', 'f_cat_0', 'f_merchant_0', 'f_brand_0', 'fp_0', 'fp_item_0', 'fp_cat_0', 'fp_merchant_0', 'fp_brand_0',
    'r_1', 'f_1', 'f_item_1', 'f_cat_1', 'f_merchant_1', 'f_brand_1', 'fp_1', 'fp_item_1', 'fp_cat_1', 'fp_merchant_1', 'fp_brand_1',
    'r_2', 'f_2', 'f_item_2', 'f_cat_2', 'f_merchant_2', 'f_brand_2', 'fp_2', 'fp_item_2', 'fp_cat_2', 'fp_merchant_2', 'fp_brand_2',
    'r_3', 'f_3', 'f_item_3', 'f_cat_3', 'f_merchant_3', 'f_brand_3', 'fp_3', 'fp_item_3', 'fp_cat_3', 'fp_merchant_3', 'fp_brand_3',
    'r_um', 'f_um', 'f_item_um', 'f_cat_um', 'f_brand_um',
    'r_um_0', 'f_um_0', 'f_item_um_0', 'f_cat_um_0', 'f_brand_um_0', 'fp_um_0', 'fp_item_um_0', 'fp_cat_um_0', 'fp_brand_um_0',
    'r_um_1', 'f_um_1', 'f_item_um_1', 'f_cat_um_1', 'f_brand_um_1', 'fp_um_1', 'fp_item_um_1', 'fp_cat_um_1', 'fp_brand_um_1',
    'r_um_2', 'f_um_2', 'f_item_um_2', 'f_cat_um_2', 'f_brand_um_2', 'fp_um_2', 'fp_item_um_2', 'fp_cat_um_2', 'fp_brand_um_2',
    'r_um_3', 'f_um_3', 'f_item_um_3', 'f_cat_um_3', 'f_brand_um_3', 'fp_um_3', 'fp_item_um_3', 'fp_cat_um_3', 'fp_brand_um_3',
    'r_m', 'f_m', 'f_user_m', 'f_item_m', 'f_cat_m', 'f_brand_m',
    'r_m_0', 'f_m_0', 'f_user_m_0', 'f_item_m_0', 'f_cat_m_0', 'f_brand_m_0', 'fp_m_0', 'fp_user_m_0', 'fp_item_m_0', 'fp_cat_m_0', 'fp_brand_m_0',
    'r_m_1', 'f_m_1', 'f_user_m_1', 'f_item_m_1', 'f_cat_m_1', 'f_brand_m_1', 'fp_m_1', 'fp_user_m_1', 'fp_item_m_1', 'fp_cat_m_1', 'fp_brand_m_1',
    'r_m_2', 'f_m_2', 'f_user_m_2', 'f_item_m_2', 'f_cat_m_2', 'f_brand_m_2', 'fp_m_2', 'fp_user_m_2', 'fp_item_m_2', 'fp_cat_m_2', 'fp_brand_m_2',
    'r_m_3', 'f_m_3', 'f_user_m_3', 'f_item_m_3', 'f_cat_m_3', 'f_brand_m_3', 'fp_m_3', 'fp_user_m_3', 'fp_item_m_3', 'fp_cat_m_3', 'fp_brand_m_3',
]
target = 'label'
X = df_train[features]
y = df_train[target].astype(float)
X_test = df_test[features]        
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_pool = Pool(X_train, y_train, cat_features=['gender'])
val_pool = Pool(X_val, y_val, cat_features=['gender'])  
model = CatBoostClassifier(iterations=10000, learning_rate=0.01, depth=6, eval_metric='AUC', random_seed=42, logging_level='Verbose', use_best_model=True)
model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=1000, verbose=100)
y_pred = model.predict_proba(X_test)[:, 1]
df_test['prob'] = y_pred
df_test[['user_id', 'merchant_id', 'prob']].to_csv(path + 'prediction.csv', index=False)  
print(path + 'prediction.csv saved!')


0:	test: 0.5531294	best: 0.5531294 (0)	total: 114ms	remaining: 18m 59s
100:	test: 0.6345947	best: 0.6346500 (99)	total: 6.57s	remaining: 10m 44s
200:	test: 0.6470507	best: 0.6470507 (200)	total: 12.7s	remaining: 10m 17s
300:	test: 0.6574444	best: 0.6574444 (300)	total: 19.1s	remaining: 10m 14s
400:	test: 0.6641674	best: 0.6641674 (400)	total: 25.3s	remaining: 10m 4s
500:	test: 0.6680875	best: 0.6680875 (500)	total: 31.4s	remaining: 9m 55s
600:	test: 0.6705589	best: 0.6705677 (598)	total: 37.5s	remaining: 9m 47s
700:	test: 0.6728902	best: 0.6728902 (700)	total: 43.8s	remaining: 9m 41s
800:	test: 0.6749333	best: 0.6749333 (800)	total: 50s	remaining: 9m 34s
900:	test: 0.6764788	best: 0.6764788 (900)	total: 56.1s	remaining: 9m 27s
1000:	test: 0.6776996	best: 0.6777029 (998)	total: 1m 2s	remaining: 9m 19s
1100:	test: 0.6789024	best: 0.6789024 (1100)	total: 1m 8s	remaining: 9m 11s
1200:	test: 0.6799081	best: 0.6799180 (1198)	total: 1m 14s	remaining: 9m 4s
1300:	test: 0.6808309	best: 0.680834

In [ ]:
# 5100:	test: 0.6858243	best: 0.6858491 (5074)	total: 4m 30s	remaining: 4m 20s
# 5200:	test: 0.6859037	best: 0.6859302 (5178)	total: 4m 36s	remaining: 4m 14s
# 5300:	test: 0.6858344	best: 0.6859302 (5178)	total: 4m 41s	remaining: 4m 9s
# 5400:	test: 0.6858755	best: 0.6859302 (5178)	total: 4m 46s	remaining: 4m 4s
# 5500:	test: 0.6858226	best: 0.6859302 (5178)	total: 4m 51s	remaining: 3m 58s
# 5600:	test: 0.6857450	best: 0.6859302 (5178)	total: 4m 57s	remaining: 3m 53s
# Stopped by overfitting detector  (500 iterations wait)

# bestTest = 0.6859301753
# bestIteration = 5178

# Shrink model to first 5179 iterations.